# File Share API - Ethan Tanen


## Introduction 

Here’s a short guide on how to implement a basic file sharing API using Node.js with an Express.js framework and a Mongo database. The minimal viable product is a piece of software that has the ability to tag and upload files to a database, search the database using tags as keywords and download files to the users file system. 


## Node.js and Express.js

Node.js is an event driven JavaScript runtime, namely for creating server environments. Arguably, NPM or Node’s Package Manager is just as impressive as Node.js itself. NPM, as mentioned on the Node.js website, is the largest ecosystem of open source libraries in the world! It makes it simple to add funtinality to an application without having to know the specificities behind the technology. 

Express.js is a Node.js web application framework that makes designing web applications a breeze. More on this later.

Here are links to the two pieces of software in question. 
<ul>
    <li> https://nodejs.org/en/about/ </li>
    <li> https://expressjs.com/ </li>
    
</ul>


## MongoDB, GridFS and MLab

MongoDB is a document database that stores information in JSON, a format familiar to most developers. If you’re unfamiliar with JSON, now is a good time to give it a quick google search. MongoDB stores complex files in BSON format, which is a binary representation of the file in question. The limitations of MongoDB include a hard cap at 16MB per document. The MVP described above specifies that any file should be able to be stored in the database. To remedy this issue, we are going to use GridFS.

GridFS parses large BSON documents into 255 kB chunks and keeps track of the chunk ordering for you, such that retrieval of the documents is as easy as a single method call! GridFS uses two collections to store files. The first collection is titled ‘chunks’ and it stores the binary chunks. The other collection is titled ‘files’, which stores information on which chunks comprise a single document. The other advantage of GridFS is the stream functions, which allow large documents to be moved between database and filesystem without having to move the entire file into memory. 

Here are links to the pieces of software in question. Click the second link and create a free account on mLab. 

<ul>
    <li> https://docs.mongodb.com/manual/core/gridfs/ </li>
    <li> https://mlab.com/ </li>
</ul>


## Environment Setup

Before beginning any Node.js project, there's a bit of initial setup that must be completed first. If you haven't installed Node.js yet, simply run the following command.

```bash
brew install node
```

Double check that Node.js and NPM were installed correctly by running, 
    
```bash
node --version
npm --version
```

Next, run the following command to begin a Node.js project. Feel free to fill out the prompts or simply click enter until the dialog completes. 

``` bash 
    npm init
```

If you look inside the current directory, you will see a new file titled package.json. If you plan to publish your project, this file includes the metadata on the project that prospective users will see on NPM's website, such as name and search tags. This file also lists the projects dependencies, so other users can run your code on their machine. The package.json file and NPM make Node.js applications highly portable between systems. 

This project has a number of dependencies that should be installed prior to writing any code. Installing dependencies is simple. Use the following format to install the modules express, fs, mongodb, gridfs-stream, multer and morgan.

```bash
npm install <package name> --save
```
<h4> Module Descriptions </h4>
<ul>
    <li> expresss - applicatin framework </li>
    <li> fs - interacting with the file system </li>
    <li> mongodb - connecting to mongo database </li>
    <li> gridfs-stream - streaming files to and from database </li>
    <li> multer - parsing multipart form data </li>
    <li> morgan - recording api activity </li>
        
</ul>

As an aside, the following tutorial will get you started on developing this api. It is not meant to walk you through the entire process. The user interface is left to the reader to complete. That being said, there is no reason the user interface must be graphical. The api could be interfaced using the command line or any number of methods. Look into express's view engines and static files documentation for this part of the project. 

## Connect to Database and Begin Server

Create a file in your project directory and title it "app.js". This is where the bulk of the code will be written. Use the following lines of code to gain access to the projects dependencies.

```javascript
const mongodb = require('mongodb')
const express = require('express')
```
Repeat this for the remaining dependencies or require them as needed. 

Now create your express app.

```javascript
const app = express()
app.listen(3000)
```

When the application is running all of the traffic on port 3000 is routed through the app. Next, create a const called URI and assign it the uri of your mongodb. The uri can be found on the databases console on mLab's website. Be sure to update the uri with your credentials. Retrieve your credentials by creating a user for your database on mLab. 

```javascript
// connect to mongo database
const URI = 'mongodb://' + process.env.USERNAME + ':' + process.env.PASSWORD + '@ds245240.mlab.com:45240/file-api'
mongo.MongoClient.connect(URI, async (err, db) => {
  if (err) return console.log(err)
  gfs = grid(db,mongo)
  console.log('connected to db')
})

```



Note the line of code that reads ```javascript gfs = grid(db, mongo)```. The gfs object is how our application will communicate with the mongodb. 

## Upload Files To Database 

Express routing offers a plethora of routing methods but for this project we're going to simply use the GET method and POST method. Most modern browsers only support GET and POST requests, so it's worth working within these constraints. As can be deduced from the names, GET requests are for retreiving information from a server and POST requests are for transfering data to the server. Naturally, the upload file endpoint will listen for post requests. Express routes are set up using the following form. 

```javascript
app.post('/upload', (req, res) => {
    // code!!
})                             
```

The application is being designed to easly port with the front end. A popular form encoding is 'multipart/form-data ', which is where multer will come into play. Update the code thusly: 

```javascript
const upload = require('multer')({dest:'upload'})
app.post('/u tpload', upload.single('file'), (req, res) => {
    // code!!
})                             
```

Now, the request object has a file field that can be accessed simply by typing ```javascript req.file```. Its worth printing out the request object before and after using multer. Parsing out the file is a little more involved than it appears. After completing this read through, you will certainly be more prepared to explore middlewares such as multer and body-parser. 

As mentioned earlier, we are going to use streams to move data between the users browser and the database. Streams are not an easy topic but with practice they begin to make more sense. I've included a link to a good document that outlines streams. What they boil down to is an array or buffer where not all the data is available at once. This makes streams great in instances where you're working under memory constraints. To upload a file to the database we need to open a read stream to the file and a write stream to the database. Afterwards, we'll use the stream function pipe to move data between the two streams. Use the fs module to create a read stream from the file and gridfs-stream to open a write stream to the database. The code will look something like this. 

```javascript
// create read stream from file
loc = path.join(__dirname,req.file.path)
readStream = fs.createReadStream(loc)

// create write stream to db w/ tags as aliases
writeStream = gfs.createWriteStream()

// pipe file to database
readStream.pipe(writeStream)
```

It's as simple as that!! Checkout the database viewer on mLab to see if it worked. You will need to have coded up some way of submiting form data to this endpoint before testing.

When writing to the database you have the option to include more options such as custom file names and aliases. I like to use the alias option to include file tags but the metadata option is just as good! The write stream looks like this after these updates:

```javascript 
writeStream = gfs.createWriteStream({
    filename: <filename>
    aliases: [<alias1, alias2 ...>]                               
}) 
```

Lastly, the client is expecting some sort of confirmation from the server. To end the client-server interaction include the following line of code:

```javascript
res.send('Success!')
```

## Conclusion 

At this point you should be equipped with the tools to complete the remainder of the implementation. Ofcourse you could stop here but wouldnt it be nice if you could download files, delete files or query the database? The link to the gridfs-stream documentation is included below as well as a few other relevent links. 

Links:
<ul>
    <li> gridfs-stream: https://github.com/aheckmann/gridfs-stream </li>
    <li> streams: https://github.com/aheckmann/gridfs-stream </li>
    <li> source code: https://github.com/ethantanen/File-Share-API </li>
    
    
</ul>